# Dashboard Metrics Calculation: Accumulated Downloads

共創分析ダッシュボードの指標（累積ダウンロード数）を計算するNotebook。


## Input Parameters
+ DATE 集計期間の始まりの日
+ DEBUG 手動実行時のみTrue(Cloud FunctionsからFalseを渡される。)
+ ENV productionかdevelopment.CFで実行する時意外はdevelopment
+ FREQUENCY 集計期間
+ PROJECT_ID 集計対象プロジェクトのプロジェクトID
+ PROJECT_CODE_NAME プロジェクトのコードネーム
+ PACKAGE_NAME 対象プロジェクトのパッケージ名
+ GOOGLE_PLAY_REPORTING_BUCKET Google playのbucket名
+ OUTPUT_BIGQUERY_PROJECT_ID 出力先のbigqueryのプロジェクトID
+ SECRET_ID_ISSUER ISSUER IDの格納されているGSMのシークレット名
+ VERSION_ID_ISSUER  SSUER IDの格納されているGSMのバージョン
+ SECRET_ID_KEY KEY_IDの格納されているGSMのシークレット名
+ VERSION_ID_KEY KEY IDの格納されているGSMのバージョン
+ SECRET_ID_PRIVATE = PRIVATE　KEYの格納されているGSMのバージョン
+ VERSION_ID_PRIVATE PRIVATE　KEYの格納されているGSMのバージョン
+ VENDOR_NUMBER App store connect から確認できるベンダー番号
+ APP_ID 各アプリに定義されているアプリID
+ PROVIDER_ID appleアカウントに紐づいたプロバイダーID
+ ISLATEST 最新の日付を対象にする場合はTrue,任意の日付を指定する場合はFalse


# Parameters

In [95]:
DATE = "2021-06-01" # @param {type: "date"}
DEBUG = True #@param {type: "boolean"} 手動実行時のみTrueにする。Cloud FunctionsからFalseを渡される。
ENV = "development"
FREQUENCY = "daily" #@param {type: "string"}
PROJECT_ID = "komtar_monet_prd" #@param {type: "string"}
PROJECT_CODE_NAME = "monet" #@param {type: "string"}
PACKAGE_NAME="jp.co.fuller.snowpeak" #@param {type: "string"}
GOOGLE_PLAY_REPORTING_BUCKET = "pubsite__rev_10325089898676068645" # google play consoleから調べられる
OUTPUT_BIGQUERY_PROJECT_ID = "fl-komtar-analytics-dashboard" # @param {type: "string"}
SECRET_ID_ISSUER = "monet_issuer_id"
VERSION_ID_ISSUER = "1"
SECRET_ID_KEY = "monet_key_id"
VERSION_ID_KEY = "1"
SECRET_ID_PRIVATE = "monet_private_key"
VERSION_ID_PRIVATE = "1"
VENDOR_NUMBER = "87701525"# @param {type: "string"}
IS_LATEST = False # @param {type:"boolean"}

# Constants

In [63]:
METRICS_NAME = "accumulated-downloads"

# Version

In [64]:
VERSION = "7"

# Authorize

In [65]:
if DEBUG:
    from google.colab import auth
    auth.authenticate_user()

# Imports

In [66]:
from google.cloud import storage
from google.cloud import secretmanager
import pandas as pd
from datetime import timedelta 
from datetime import datetime
import requests
import jwt
import time
import io
import gzip
import re

# Get Input Datasets

## Get Year and month

In [67]:
gs = storage.Client(project=PROJECT_ID)
blobs = gs.list_blobs(GOOGLE_PLAY_REPORTING_BUCKET,prefix=f"stats/installs/installs_{PACKAGE_NAME}_")
google_play_reporting_files = list(blobs)
first_reporting_month = re.sub("\\D","",google_play_reporting_files[0].name)
start_DATE = first_reporting_month + "01"
start_DATE = datetime.strptime(start_DATE,"%Y%m%d").strftime("%Y-%m-%d")
start_DATE

'2018-02-01'

In [68]:
tdatetime = datetime.strptime(DATE, '%Y-%m-%d')
end_DATE = (tdatetime - timedelta(days=1)).strftime('%Y-%m-%d')
end_DATE

'2021-05-31'

## Google play store

In [69]:
def getGooglePlayInstalls(file_name, date):
  gs = storage.Client(project=PROJECT_ID)
  month = f"{date.split('-')[0]}{date.split('-')[1]}"
  blob_address = f"stats/installs/installs_{PACKAGE_NAME}_{month}_{file_name}"
  blob = gs.bucket(GOOGLE_PLAY_REPORTING_BUCKET).blob(blob_address)
  data = blob.download_as_string()
  return pd.read_csv(io.StringIO(data.decode("utf-16")))

In [70]:
periods = pd.date_range(start=start_DATE, end=end_DATE, freq = "MS")
periods

DatetimeIndex(['2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01',
               '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-01',
               '2018-10-01', '2018-11-01', '2018-12-01', '2019-01-01',
               '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01',
               '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
               '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01',
               '2020-02-01', '2020-03-01', '2020-04-01', '2020-05-01',
               '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01',
               '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01',
               '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01'],
              dtype='datetime64[ns]', freq='MS')

In [71]:
df_googleplay_installs = pd.DataFrame()
for period in periods:
  date_T = period.strftime('%Y-%m-%d')
  df_googleplay_installs = pd.concat([df_googleplay_installs, getGooglePlayInstalls(file_name = "overview.csv", date = date_T)])
df_googleplay_installs.rename(columns={"Date":"date"}, inplace=True)
df_googleplay_installs

,date,Package Name,Daily Device Installs,Daily Device Uninstalls,Daily Device Upgrades,Total User Installs,Daily User Installs,Daily User Uninstalls,Active Device Installs,Install events,Update events,Uninstall events
0,2018-02-24,jp.co.fuller.snowpeak,1,1,0,1,1,1,0,1,0,1
1,2018-02-25,jp.co.fuller.snowpeak,11,0,1,12,11,0,12,14,0,2
2,2018-02-26,jp.co.fuller.snowpeak,27,2,0,37,25,2,37,27,0,2
3,2018-02-27,jp.co.fuller.snowpeak,33,1,1,70,33,1,69,33,1,1
4,2018-02-28,jp.co.fuller.snowpeak,727,4,47,791,719,4,793,733,46,11
...,...,...,...,...,...,...,...,...,...,...,...,...
26,2021-05-27,jp.co.fuller.snowpeak,45,0,0,0,28,26,36567,50,45,25
27,2021-05-28,jp.co.fuller.snowpeak,77,0,0,0,49,39,34717,89,56,51
28,2021-05-29,jp.co.fuller.snowpeak,102,0,0,0,69,26,34782,108,47,33
29,2021-05-30,jp.co.fuller.snowpeak,97,0,0,0,64,23,34787,104,44,34


## Apple appstore

### Get Authonication Info from GCP Secret Manager

In [96]:
client = secretmanager.SecretManagerServiceClient()

#### Issuer id

In [97]:
name = f"projects/{OUTPUT_BIGQUERY_PROJECT_ID}/secrets/{SECRET_ID_ISSUER}/versions/{VERSION_ID_ISSUER}"
response = client.access_secret_version(request={"name": name})
issuer_id = response.payload.data.decode("UTF-8")

#### Key id

In [98]:
name = f"projects/{OUTPUT_BIGQUERY_PROJECT_ID}/secrets/{SECRET_ID_KEY}/versions/{VERSION_ID_KEY}"
response = client.access_secret_version(request={"name": name})
kid= response.payload.data.decode("UTF-8")

#### Private key

In [99]:
name = f"projects/{OUTPUT_BIGQUERY_PROJECT_ID}/secrets/{SECRET_ID_PRIVATE}/versions/{VERSION_ID_PRIVATE}"
response = client.access_secret_version(request={"name": name})
private_key = response.payload.data.decode("UTF-8")

In [107]:
token = jwt.encode(
    payload={
        "iss": issuer_id,
        "exp": int(time.time()) + 20*60,
        "aud": "appstoreconnect-v1"
    },
    key=private_key,
    algorithm="ES256",
    headers={
        "alg" : "ES256",
        "kid": kid,
        "typ": "JWT"        
    }
)

In [ ]:
df_sales_reports = pd.DataFrame()
for target_date in pd.date_range(start_DATE,end_DATE,freq="D"):
  report_date = target_date.strftime("%Y-%m-%d")
  url = f"https://api.appstoreconnect.apple.com/v1/salesReports?filter[frequency]=DAILY&filter[vendorNumber]={VENDOR_NUMBER}&filter[reportDate]={report_date}&filter[reportSubType]=SUMMARY&filter[reportType]=SALES"
  header = {'Authorization': 'Bearer {}'.format(token)}
  responce = requests.get(url, headers=header)
  if(responce.status_code==200):
    gzip_file = io.BytesIO(responce.content)
    with gzip.open(gzip_file, "rt") as f:
      data = f.read()
    df_sales_reports = pd.concat([df_sales_reports,pd.read_csv(io.StringIO(data),sep="\t")]) 
#df_sales_reports = df_sales_reports[df_sales_reports["SKU"]==PACKAGE_NAME]
df_sales_reports["Product Type Identifier"] = df_sales_reports["Product Type Identifier"].astype(str)
df_sales_reports = df_sales_reports[df_sales_reports["Product Type Identifier"].str.contains("1|3",na=False)].reset_index(drop=True) 
df_sales_reports

# Calculate accumulated installs

In [121]:
df_appstore_installs = df_sales_reports.groupby(["Begin Date"]).sum().reset_index()[["Begin Date","Units"]]
df_appstore_installs.columns = ["date","installs"]
df_appstore_installs["date"] = pd.to_datetime(df_appstore_installs["date"])
df_appstore_installs["date"] = df_appstore_installs["date"].dt.strftime("%Y-%m-%d")

In [122]:
df_installs = df_appstore_installs.merge(df_googleplay_installs, on="date")
df_installs.rename(columns={
    "Package Name": "package_name",
    "installs": "appstore_installs", 
    "Daily Device Installs": "googleplay_installs"
}, inplace=True)
df_installs = df_installs[["date", "appstore_installs", "googleplay_installs"]]
df_installs = df_installs.sort_values("date")
df_installs['installs'] = df_installs['appstore_installs'] + df_installs['googleplay_installs']
df_installs["accumulated_appstore_installs"] = df_installs["appstore_installs"].cumsum()
df_installs["accumulated_googleplay_installs"] = df_installs["googleplay_installs"].cumsum()
df_installs['accumulated_installs'] = df_installs['accumulated_appstore_installs'] + df_installs['accumulated_googleplay_installs']
df_installs = df_installs.reset_index(drop=True)
df_installs

,date,appstore_installs,googleplay_installs,installs,accumulated_appstore_installs,accumulated_googleplay_installs,accumulated_installs
0,2020-06-10,283,47,330,283,47,330
1,2020-06-11,288,53,341,571,100,671
2,2020-06-12,261,93,354,832,193,1025
3,2020-06-13,529,117,646,1361,310,1671
4,2020-06-14,634,72,706,1995,382,2377
...,...,...,...,...,...,...,...
351,2021-05-27,359,45,404,174070,27047,201117
352,2021-05-28,368,77,445,174438,27124,201562
353,2021-05-29,602,102,704,175040,27226,202266
354,2021-05-30,733,97,830,175773,27323,203096


# Output

In [123]:
output_columns = ["date", "accumulated_appstore_installs", "accumulated_googleplay_installs", "accumulated_installs"]
df_installs["date"] = pd.to_datetime(df_installs["date"]).dt.date
df_installs['appstore_installs'] = df_installs['appstore_installs'].astype('int')
df_installs['accumulated_appstore_installs'] = df_installs['accumulated_appstore_installs'].astype('int')
df_installs['accumulated_installs'] = df_installs['accumulated_installs'].astype('int')
gbq_output_path = f"""{PACKAGE_NAME.replace(".","_")}_{METRICS_NAME.replace("-","_")}.{FREQUENCY}_events"""
df_installs[output_columns].to_gbq(gbq_output_path,
                                   if_exists="replace",
                                   table_schema=[{'name': 'date','type': 'DATE'},
                                                 {'name': 'appstore_installs','type': 'INT64'},
                                                 {'name': 'googleplay_installs','type': 'INT64'},
                                                 {'name': 'accumulated_appstore_installs	','type': 'INT64'},
                                                 {'name': 'accumulated_googleplay_installs	','type': 'INT64'},
                                                 {'name': 'accumulated_installs	','type': 'INT64'}],
                                   project_id=OUTPUT_BIGQUERY_PROJECT_ID)
df_installs

1it [00:05,  5.68s/it]


,date,appstore_installs,googleplay_installs,installs,accumulated_appstore_installs,accumulated_googleplay_installs,accumulated_installs
0,2020-06-10,283,47,330,283,47,330
1,2020-06-11,288,53,341,571,100,671
2,2020-06-12,261,93,354,832,193,1025
3,2020-06-13,529,117,646,1361,310,1671
4,2020-06-14,634,72,706,1995,382,2377
...,...,...,...,...,...,...,...
351,2021-05-27,359,45,404,174070,27047,201117
352,2021-05-28,368,77,445,174438,27124,201562
353,2021-05-29,602,102,704,175040,27226,202266
354,2021-05-30,733,97,830,175773,27323,203096
